In [1]:
import sys
import numpy as np
from matplotlib import pyplot as plt
import scipy.signal
sys.path.append('./src')
sys.path.append('/usr/lib/python3/dist-packages')
print(sys.path)
from msg.msg import MSGNet
import os
os.environ["CC"] = "gcc"
os.environ["CXX"] = "gcc"

import imageio
import torch
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
torch.cuda.empty_cache()

import scipy.interpolate
%matplotlib inline

from msg.utils import modcrop, normalize, resample, rgb2ycbcr

if 'ipykernel' in sys.modules:
    from tqdm import tqdm_notebook as tqdm
else:
    from tqdm import tqdm as tqdm
# from pix_transform.pix_transform import PixTransform
# from baselines.baselines import bicubic
# from utils.utils import downsample,align_images
# from prox_tv import tvgen
# from utils.plots import plot_result


['/Users/felicity/Desktop/MSG_TOPO', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python37.zip', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages', '/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/extensions', '/Users/felicity/.ipython', './src', '/usr/lib/python3/dist-packages']


### Get data

In [2]:
directory = os.path.dirname(os.getcwd())
directory

'/Users/felicity/Desktop'

In [3]:
directory = '/Users/felicity/Desktop/Super-resolution-drones-depth-images/ICCV2019'
rgb_dir = directory+'/EPFL_10_newsample_nadir/rgb/'
_files = os.listdir(rgb_dir)
_files.sort()
_rgb_files = [rgb_dir + f for f in _files]
_rgb_files.sort()
print(len(_rgb_files))

dist_dir = directory+'/EPFL_10_newsample_nadir/dist/'
_files = os.listdir(dist_dir)
_files.sort()
_dist_files = [dist_dir + f for f in _files]
_dist_files.sort()
print(len(_dist_files))

25
25


In [4]:
####  define parameters  ########################################################
params = {'img_idxs' : [], # idx images to process, if empty then all of them
            
          'scaling': 40,
          'greyscale': False, # Turn image into grey-scale
          'channels': -1,
          
          'spatial_features_input': True,
          'weights_regularizer': [0.0001, 0.001, 0.0001], # spatial color head
          'loss': 'l1',
 
          'optim': 'adam',
          'lr': 0.001,
                  
          'batch_size': 32,
          'iteration': 1024*12*18//32,
                  
          'logstep': 64,
          
          'final_TGV' : False, # Total Generalized Variation in post-processing
          'align': False, # Move image around for evaluation in case guide image and target image are not perfectly aligned
          'delta_PBP': 1, # Delta for percentage of bad pixels 
         }

In [5]:
def downsample(ar, factor):
    kernel = np.full((factor, factor), 1 / (factor**2))
    ar = scipy.signal.convolve2d(np.asarray(ar), kernel, mode='full')
    ar = ar[factor-1::factor, factor-1::factor]
    return ar

In [6]:
guide_imgs = []
target_imgs= []
source_imgs= []

for num in range(len(_rgb_files)):
    rgb_img = imageio.imread(_rgb_files[num])
    guide_imgs.append(rgb_img)
    dist_img = torch.load(_dist_files[num])
    dist_img = dist_img.detach().cpu().numpy()
    target_imgs.append(dist_img)
    source_imgs.append(downsample(dist_img,params['scaling']))
        
guide_imgs = np.asarray(guide_imgs)
# guide_imgs = np.einsum('ijkl->iljk', guide_imgs)        
target_imgs = np.asarray(target_imgs)
source_imgs = np.asarray(source_imgs)

print(guide_imgs.shape)
print('max value in the image is: ' + str(guide_imgs.max()))
print('min value in the image is: ' + str(guide_imgs.min()))
print(target_imgs.shape)
print('max value in the image is: ' + str(target_imgs.max()))
print('min value in the image is: ' + str(target_imgs.min()))
print(source_imgs.shape)
print('max value in the image is: ' + str(target_imgs.max()))
print('min value in the image is: ' + str(target_imgs.min()))

(25, 480, 720, 3)
max value in the image is: 255
min value in the image is: 0
(25, 480, 720)
max value in the image is: 97.91275
min value in the image is: 44.26991
(25, 12, 18)
max value in the image is: 97.91275
min value in the image is: 44.26991


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
guide_imgs = torch.from_numpy(guide_imgs).float().to(device)
source_imgs = torch.from_numpy(source_imgs).float().to(device)
target_imgs = torch.from_numpy(target_imgs).float().to(device)

In [8]:
train_data = torch.utils.data.TensorDataset(guide_imgs, source_imgs, target_imgs)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)

### Train the model

In [9]:
mynet

NameError: name 'mynet' is not defined

In [10]:
# define the optimization
mynet = MSGNet(40).train().to(device)
#     mynet = PixTransformNet(channels_in=guide_img.shape[0],
#                             weights_regularizer=params['weights_regularizer']).train().to(device)
optimizer = optim.Adam(mynet.parameters(), lr=params['lr'])
if params['loss'] == 'mse':
    myloss = torch.nn.MSELoss()
elif params['loss'] == 'l1':
    myloss = torch.nn.L1Loss()
else:
    print("unknown loss!")

epochs = 100
mse_values = []
mae_values = []
with tqdm(range(0, epochs), leave=True) as tnr:
    tnr.set_postfix(consistency=-1.)
    for epoch in tnr:
        running_loss = 0.0
        loss_values = []
        
        # x = guide (rgb), y = source (lr), z = target (hr) 
        for (x, y, z) in train_loader:
            optimizer.zero_grad()

            y_pred = mynet(y,x)

            train_loss = myloss(y_pred, z)

            train_loss.backward()
            optimizer.step()

            loss_values.append(train_loss.cpu().detach().numpy())
    loss_values_array = np.array(loss_values)
    print(loss_values_array.shape)
    print(loss_values_array)
    plt.figure()
    plt.plot(loss_values_array)
    plt.title("loss during training according to " + params['loss'] + " loss parameter")

#     mse_values_array = np.array(mse_values)
#     print("----------MSE----------")
#     print(mse_values_array.shape)
#     print(mse_values_array)
#     plt.figure()
#     plt.plot(mse_values_array)
#     plt.title("MSE loss, when target image is given")

#     mae_values_array = np.array(mae_values)
#     print("----------MAE----------")
#     print(mae_values_array.shape)
#     print(mae_values_array)
#     plt.figure()
#     plt.plot(mae_values_array)
#     plt.title("MAE loss, when target image is given")

# save the trained network
save_path = directory+'/saved_model_MSG_temp'+str(constant + idx)+'.pt'
# check if the directory exist
os.makedirs(os.path.dirname(save_path),exist_ok=True)
torch.save(mynet.state_dict(), save_path)    



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


  0%|          | 0/100 [00:00<?, ?it/s]

RuntimeError: filter weights array has incorrect shape.

In [ ]:
# compute final prediction, un-normalize, and back to numpy
mynet.eval()
predicted_target_img = mynet(guide_img).squeeze()
predicted_target_img = source_img_mean + source_img_std * predicted_target_img
predicted_target_img = predicted_target_img.cpu().detach().squeeze().numpy()


In [ ]:
# some random code
# define the optimization
criterion = MSELoss()
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
# enumerate epochs
for epoch in range(100):
    # enumerate mini batches
    for i, (inputs, targets) in enumerate(train_dl):
    	...
# clear the gradients
optimizer.zero_grad()
# compute the model output
yhat = model(inputs)
# calculate loss
loss = criterion(yhat, targets)
# credit assignment
loss.backward()
# update model weights
optimizer.step()
#show the output of the model
# convert row to data
row = Variable(Tensor([row]).float())
# make prediction
yhat = model(row)
# retrieve numpy array
yhat = yhat.detach().numpy()

In [ ]:
### Test the model
model = MSGNet(4)
model.load_state_dict(torch.load('checkpoints/x4_clean.pth')['model_state_dict'])
for sample in data.keys():
    data[sample]['sr4_disparity'] = model.test(data[sample]['lr4_disparity'], data[sample]['rgb'])

